# AUGMENTED REALITY OBJECT INSERTION

## SETTING UP THE SYSTEM AND LIBRARIES

In [ ]:
!pip install opencv-python

In [ ]:
import os
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from Camera_Calibration import *
from Homographies import *
from Object_Loader import *

## CALIBRATING THE CAMERA TO FIND THE INTRINSIC MATRIX

In [ ]:
num_corners = (10, 7)

file_loc = 'Resources/Calibration/'
int_mat = calibrate_camera(num_corners,file_loc)

In [ ]:
print(int_mat)

## COMPUTING HOMOGRAPHS

In [ ]:
im1 = 'Resources/Reference/base1.jpg'
ref = 'Resources/Reference/surroundings.jpg'

homography_matrix = compute_homography(im1,ref)

In [ ]:
print(homography_matrix)

## CHECKING IF CAMERA IS WORKING OR NOT ON LOCAL MACHINE

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
once = False
while True:
    # read the current frame
    ret, frame = cap.read()
    if not ret:
        if not once:
            print("Unable to capture video")
            once = True
    else:
        if not once:
            print("Capturing video successfully")
            once = True
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## MAIN CODE

In [ ]:
homography = None

In [ ]:
# Create a SIFT object
sift = cv2.SIFT_create()

In [ ]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [ ]:
base = cv2.imread(os.path.join('Resources/Reference/base1.jpg'), 0)
base = cv2.resize(base, (int(base.shape[1]/4), int(base.shape[0]/4)))

In [ ]:
kp_base, des_base = sift.detectAndCompute(base, None)

In [ ]:
obj = Object('Resources/Models/model2.obj', swap_y_z=True)

In [ ]:
def define_points(vert, face_vert, sc_mat , wi, he):
    points = np.array([vert[vertex - 1] for vertex in face_vert])
    points = np.dot(points, sc_mat)
    points = np.array([[p[0] + wi / 2, p[1] + he / 2, p[2]] for p in points])
    return points

def render(img, object_3d, projection_mat, model):

    vertices = object_3d.vertices
    scale_matrix = np.eye(3) * 3
    height, width = model.shape

    for face in object_3d.faces:
        face_vertices = face[0]
        points = define_points(vertices, face_vertices, scale_matrix, height, width)
        img_points = np.int32(cv2.perspectiveTransform(points.reshape(-1, 1, 3), projection_mat))
        cv2.fillConvexPoly(img, img_points, (128, 128, 128))

    return img

In [ ]:
def get_projection_matrix(camera_parameters, homograph):

    homograph = homograph * (-1)

    transformation = np.dot(np.linalg.inv(camera_parameters), homograph)

    # normalise vectors
    l = math.sqrt(np.linalg.norm(transformation[:, 0], 2) * np.linalg.norm(transformation[:, 1], 2))
    rot_1 = transformation[:, 0] / l
    rot_2 = transformation[:, 1] / l
    translation = transformation[:, 2] / l

    # compute the orthonormal basis
    c = rot_1 + rot_2
    p = np.cross(rot_1, rot_2)
    d = np.cross(c, p)
    rot_1 = np.dot(c / np.linalg.norm(c, 2) + d / np.linalg.norm(d, 2), 1 / math.sqrt(2))
    rot_2 = np.dot(c / np.linalg.norm(c, 2) - d / np.linalg.norm(d, 2), 1 / math.sqrt(2))
    rot_3 = np.cross(rot_1, rot_2)

    # compute the 3D projection matrix
    proj = np.stack((rot_1, rot_2, rot_3, translation)).T

    return np.dot(camera_parameters, proj)

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
MIN_MATCHES = 10

In [ ]:
while True:
    # read the current frame
    ret, frame = cap.read()
    if not ret:
        print("Unable to capture video")
        # find and draw the keypoints of the frame
    kp_frame, des_frame = sift.detectAndCompute(frame, None)

    # match frame descriptors with model descriptors
    matches = flann.knnMatch(des_base, des_frame, k=2)

    # store all the good matches as per Lowe's ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good.append(m)

    if len(good) > MIN_MATCHES:

        # Compute source points
        src_pts = np.float32([kp_base[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        # Compute homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Make a rectangle over the base
        h, w = base.shape
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, homography)
        frame = cv2.polylines(frame, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

        # render 3d model if homography is found
        if homography is not None:
            projection = get_projection_matrix(int_mat, homography)
            frame = render(frame, obj, projection, base)
        else:
            continue

        # show 10 good matches
        frame = cv2.drawMatches(base, kp_base, frame, kp_frame, good[0:10], None, flags=2)

        # show result
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        homography = None
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")

cap.release()
cv2.destroyAllWindows()